In [11]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import re
from bs4 import BeautifulSoup

In [9]:
# Examples from the dataset
!cat ex6/emailSample1.txt

> Anyone knows how much it costs to host a web portal ?
>
Well, it depends on how many visitors you're expecting.
This can be anywhere from less than 10 bucks a month to a couple of $100. 
You should checkout http://www.rackspace.com/ or perhaps Amazon EC2 
if youre running something big..

To unsubscribe yourself from this mailing list, send an email to:
groupname-unsubscribe@egroups.com



In [10]:
!cat ex6/emailSample2.txt

Folks,
 
my first time posting - have a bit of Unix experience, but am new to Linux.

 
Just got a new PC at home - Dell box with Windows XP. Added a second hard disk
for Linux. Partitioned the disk and have installed Suse 7.2 from CD, which went
fine except it didn't pick up my monitor.
 
I have a Dell branded E151FPp 15" LCD flat panel monitor and a nVidia GeForce4
Ti4200 video card, both of which are probably too new to feature in Suse's default
set. I downloaded a driver from the nVidia website and installed it using RPM.
Then I ran Sax2 (as was recommended in some postings I found on the net), but
it still doesn't feature my video card in the available list. What next?
 
Another problem. I have a Dell branded keyboard and if I hit Caps-Lock twice,
the whole machine crashes (in Linux, not Windows) - even the on/off switch is
inactive, leaving me to reach for the power cable instead.
 
If anyone can help me in any way with these probs., I'd be really grateful -
I've searched the 'ne

In [13]:
import nltk, nltk.stem.porter
def processEmail(email):
    """Email preprocessing and normalization"""
    
    # Convert to lower-case
    email = email.lower()
    
    # Strip HTML
    email = BeautifulSoup(email, 'html.parser').get_text()
    
    # Normalize numbers
    email = re.sub('[0-9]+','number',email)
    
    # Normalize URLs
    email = re.sub('(http|https)://[^\s]*', 'httpaddr', email)
    
    # Normalize emails
    email = re.sub('[^\s]+@[^\s]+', 'emailaddr', email)
    
    # Normalize dollars
    email = re.sub('[$]+', 'dollar', email)
    
    return email

def tokenizeEmail(email):
    """
    Convert email to token list.
    """
    # Pre-processing
    email = processEmail(email)
    
    # Split into tokens
    tokens = re.split('[ \@\$\/\#\.\-\:\&\*\+\=\[\]\?\!\(\)\{\}\,\'\"\>\_\<\;\%]', email)
    
    # Use porter stemmer to stem individual words in the email
    # Stemmers remove derivational afixes.
    stemmer = nltk.stem.porter.PorterStemmer()
    
    for word in tokens:
        # Remove non-alphanumeric characteers
